In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from time import clock

In [2]:
begin_page = 2001
end_page = 3000
master_page = 'https://www.redkaraoke.com/karaoke/celine-dion/my-heart-will-go-on/11827/recordings/p'
basePlayerUrl = "https://www.redkaraoke.com";
baseS3Url = "https://s3.amazonaws.com/";
#js file: https://d4rgvppl55fks.cloudfront.net/es/js/playerRecs.js

In [3]:
links_list = []
for i in range(begin_page, end_page, 1):
    r = requests.get(master_page + str(i))
    soup = BeautifulSoup(r.content, 'html.parser')
    for link in soup.find_all('a', attrs = {'class':'result result-recording'}):
        recording_link = 'https://www.redkaraoke.com' + link.get('href')
        links_list.append(recording_link)
#         print(recording_link)
        
# Remove duplicated links
links_list_unique = set(links_list)

In [4]:
print(len(links_list))
print(len(links_list_unique))

23976
1022


In [5]:
def extractUserRecording(soup):
# Localize data in "userRecording" object
    pattern = re.compile(r"userRecording")
    data = soup.find('script', text=pattern)

    pattern = re.compile(r"userRecording\.([a-zA-Z0-9]+)=([^;]+);")
    data = soup.find('script', text=pattern)
    ur_string = re.findall(r"userRecording\.([a-zA-Z0-9]+)=([^;]+);",str(data))
    ur_dict = dict(ur_string)
    for key in ur_dict.keys():
        ur_dict[key] = ur_dict[key][1:-1]
    return ur_dict 

In [6]:
def getDownloadLink(ur_dict):
    isDueto = False

    if 'isDueto' in ur_dict.keys(): 
        isDueto = True
    else:
        isDueto = False

    if ur_dict['file'][-1] == '3':
        file_type = 'mp3'
        if ur_dict['lang'] == 'es':
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/mp3/es/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraoke/data/mp3/"
            else:
                filepath = basePlayerUrl + "/data/mp3/es/"
        elif ur_dict['lang'] == 'jp':
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/mp3/jp/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraokejp/data/mp3/"
            else:
                filepath = baseS3Url + "/data/mp3/jp/"
        else:
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/mp3/en/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraokecom/data/mp3/"
            else:
                filepath = basePlayerUrl + "/data/mp3/en/"
    else:
        file_type = 'vid'
        if ur_dict['lang'] == 'es':
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/vid/es/"
            elif 'isDueto' in ur_dict.keys():
                filepath = baseS3Url + "redkaraoke/data/vid/"
            else:
                filepath = basePlayerUrl + "/data/vid/es/"
        elif ur_dict['lang'] == 'jp':
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/vid/jp/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraokejp/data/vid/"
            else:
                filepath = baseS3Url + "/data/vid/jp/"
        else:
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/vid/en/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraokecom/data/vid/"
            else:
                filepath = basePlayerUrl + "/data/vid/en/"
    download_link = filepath + ur_dict['file']
    return download_link, file_type, isDueto

In [7]:
counter = 0

#Initialize lists
view_list = []
like_list = []
comment_list = []
recording_list = []
follower_list = []
following_list = []
date_list = []

location_list = []
user_avatar_list = []
user_group_list = []
user_name_list = []
language_list = []
song_title_list = []
artist_list = []

download_link_list = []
recording_link_list = []
file_type_list = []
is_duet_list = []
gender_list = []
country_list = []
recording_id_list = []

# Send request for each link in links_list_unique
for link_sub in links_list_unique:
    r = requests.get(link_sub)
    soup = BeautifulSoup(r.content, 'lxml')
    
    try:
        counter = counter + 1
        
        # Extract information from html code:
        view = int(soup.find('li', attrs = {'class':'rec-views'}).text)
        like = soup.find('li', attrs = {'class':'rec-likes'}).text
        comment = int(soup.find('li', attrs = {'class':'rec-comments'}).text)
        recording = int(soup.find('li', attrs = {'class':'tam1'}).find('strong').text)
        follower = int(soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[5].find('strong').text)
        following = (soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[6].find('strong').text)
        date = soup.find_all('div', attrs = {'class':'infoTema_textoBlanco'})[2].text[3:]
        gender_loc = soup.find('div', attrs = {'class':'userdata'}).find('p').text
        
        if gender_loc.split(',')[0] == 'Man':
            gender = 'man'
        elif gender_loc.split(',')[0] == 'Woman':
            gender = 'woman'
        else:
            gender = float('nan')
        country = gender_loc.split()[-1]
            
        recording_link = link_sub

        ur_dict = extractUserRecording(soup)
        download_link, file_type, is_duet = getDownloadLink(ur_dict)
        print(download_link)
        
        location = ur_dict['loc']
        user_avatar = ur_dict['uavatar']
        user_group = ur_dict['ugroup']
        user_name = ur_dict['u']
        language = ur_dict['lang']
        song_title = ur_dict['title']
        artist = ur_dict['a']
        recording_id = ur_dict['rid']
                
    except (IndexError, AttributeError) as e:
        continue
        
    view_list.append(view)
    like_list.append(like)
    comment_list.append(comment)
    recording_list.append(recording)
    follower_list.append(follower)
    following_list.append(following)
    date_list.append(date)    
    gender_list.append(gender)
    country_list.append(country)
    
    location_list.append(location)
    user_avatar_list.append(user_avatar)
    user_group_list.append(user_group)
    user_name_list.append(user_name)
    language_list.append(language)
    song_title_list.append(song_title)
    artist_list.append(artist)
    download_link_list.append(download_link)
    recording_link_list.append(recording_link)
    file_type_list.append(file_type)
    is_duet_list.append(is_duet)
    recording_id_list.append(recording_id)
    print(counter)

https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/82586abdb7a6b016d95a83de36a6f56e_1.mp3
1
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/6145a5b40d2aaaf96606ff187fe67be7_1.mp4
2
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/d6abaed0d4904eddabf721882246679d_1.mp3
3
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/ce6b0dbb253a1be961f37945f0891710_1.mp4
4
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/87b4de0dce8bb3e26daa4a67a6fb42f9_1.mp3
5
https://www.redkaraoke.com/data/mp3/es/11827/457dfb559ae98225e3ac8a3a0268bc9b_1.mp3
6
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/c7ddbef01feeaf978b4c53808b41412e_1.mp3
7
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/37500af998dbac1db77d0192bb369c24_1.mp4
8
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/33289bd7365014d3676ef6fae7ec7dee_1.mp4
9
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/afccdbc568693dab78adeb4aadf0f614_1.mp4
10
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/9096f19

https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/61fe13d9f14b521c5e69071dfd1c145d_1.mp3
87
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/8b21f1ca51b585218f3cea7317b9695f_1.mp4
88
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/a5d9eeeca069179b3847754a8dd542cf_1.mp3
89
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/a9d5acd01956c311170bd7f9fe493c0a_1.mp3
90
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/a10bd5aba6b82fa8b55c05ef8212cd53_1.mp4
91
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/ecf75d320454585ac480f350f688d1e7_1.mp4
92
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/f3d60ea8536986498e552bf3316c3329_1.mp4
93
https://www.redkaraoke.com/data/mp3/es/11827/94de63c2066c2cdfa4df4609bc67f0b6_1.mp3
94
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/daa16f9e0ed31b311a7bbbeb6ad94a80_1.mp3
95
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/1839820b0d12ac9fe85249234574e47d_1.mp3
96
https://s3.amazonaws.com/redkaraokecom/data/mp3/1

https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/776a8e57cf707e6715deb315899bfc19_1.mp3
175
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/e8648b51d4bf2491ec02943204d277f3_1.mp4
176
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/2fbe49e3470c03a56aa0f51a65f1b480_1.mp4
177
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/536f7ea4c5a81d15cdde243df705ff8b_1.mp4
178
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/46017a1cb1ad54203dce740389e1c8fd_1.mp4
179
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/2cafad6c237a2b34fdb6290be618ef21_1.mp4
180
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/68945d709c507a63b7a7f90aeeb96263_1.mp4
181
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/8783ad2262ae9655529554e138bb4537_1.mp3
182
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/d6f6f2c89d5e7565a97919f994460bbf_1.mp3
183
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/5d02a57e225868013eff2c395848905d_1.mp4
184
https://s3.amazonaws.com/redka

https://s3.amazonaws.com/redkaraokecom/data/vid/11827/fe3bf47b4dd58e97404c652e34cf6a8e_1.mp4
261
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/4e9f780eee10006b1474901890de4d49_1.mp3
262
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/a10a51342cc6a7a5054c9a361362dd57_1.mp4
263
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/c34ce3d02f30ad27b70c9ee0f6189a98_1.mp4
264
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/1fc23d569eebaaff061150decbb77ea0_1.mp4
265
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/0bcd4d731d4121b8f78e54c19bccc31e_1.mp3
266
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/8c62d1040159000eeafc859da45fd2de_1.mp3
267
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/78708e33f6b5e72a1bdb4792acdeb015_1.mp4
268
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/884fb43fa6a11dd777a536e180de6109_1.mp4
269
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/40556f2ca4eadc4420e6fa52b166be95_1.mp4
270
https://s3.amazonaws.com/redka

https://s3.amazonaws.com/redkaraokecom/data/vid/11827/a121a5999c00b0143a7801d54b7784ad_1.mp4
347
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/a5169ef91123934fa7dffb00c91e6147_1.mp4
348
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/9909f9724284f85e51d166c699527c25_1.mp4
349
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/f1d767a64a90b504b8ebe0130f750899_1.mp4
350
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/3a9c383aa75882c9dae61121e8d6d7c5_1.mp4
351
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/4d009d6c547e5323858fffc0b66b54e4_1.mp4
352
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/6da987d79a4cf6dd67906346d70bfbb2_1.mp3
353
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/9f2b60a00c2a77f3658517fdd143d10c_1.mp4
354
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/41ce74c402ae4646fe5b3063ce363667_1.mp4
355
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/260ff55a6c8ddd75452b0f6797ac78ed_1.mp4
356
https://s3.amazonaws.com/redka

...

https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/363aefd0856bb533dec5915a688d9abc_1.mp3
608
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/7068596319fdcdeef02713546e52f56c_1.mp4
609
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/b6ffdda05a1cd3361c15c29ec18de7ff_1.mp4
610
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/88d9cb07ce9f43ea833f2da79169df6c_1.mp4
611
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/b6d3115b0fe7af8be7b78e39bba11c32_1.mp3
612
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/27292f1242ea0452f0b6b4e231162c65_1.mp3
613
https://www.redkaraoke.com/data/mp3/en/11827/b505e37631fd4680805702b756da69cb_1.mp3
614
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/ee9a4ba5bb1581453de95076e2792c97_1.mp4
615
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/d4617249d96f888aecd48d843b5eabeb_1.mp3
616
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/e3a67e64cab9396c0e56202af63b6077_1.mp4
617
https://s3.amazonaws.com/redkaraokecom/

https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/5526a7b30daf607e4f0c58eede92a207_1.mp3
695
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/173f217d6caa9f89e5fe5c8d30c7c681_1.mp4
696
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/5919d641116c4087a1673d3c80edea81_1.mp3
697
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/437d5dc3215489671e99b23ad9a6ffd9_1.mp3
698
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/42eacaf1b9214aa5cf5d38d2251d0c64_1.mp3
699
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/996298e3b0fafd0825f1044f236ca834_1.mp3
700
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/086b5fea6eccdde427f4ca33020b9b43_1.mp4
701
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/26fffce3095e5d66757510597fcd2a6b_1.mp3
702
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/593126b50388ea510fffec37ef2e35bf_1.mp4
703
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/b1e648d7aa01662246c2b6ab1300f5e9_1.mp3
704
https://s3.amazonaws.com/redka

https://www.redkaraoke.com/data/vid/en/11827/64d20a9206606f35defea117a3f174a4_1.mp4
780
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/7960acda5cecbbaae42253e8ab876155_1.mp3
781
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/ac59f7e7fd0ce1c3bb918a9aee1d2c24_1.mp4
782
https://s3.amazonaws.com/redkaraokejp/data/mp3/11827/d966ab7edfecce2d77c9b27163bf5913_1.mp3
783
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/e21ed71cd201dfb1f2e577645114ffe9_1.mp4
784
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/2d3d4acd65d03b75c04ed86bb584731b_1.mp3
785
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/26647a5c3d4ce2a7180a0df602928dd7_1.mp3
786
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/46a302d1412eed2c7caff90d38ae01cf_1.mp3
787
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/75a35b38ee5977ede4e4f318552c7d3b_1.mp3
788
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/5a8912e0c27d32cc6cf37ddfcdf6ad85_1.mp3
789
https://s3.amazonaws.com/redkaraokecom/data

https://s3.amazonaws.com/redkaraokecom/data/vid/11827/3d2b73f4df2d7e5516c86c44dc06bc0d_1.mp4
866
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/ed4a4406bb01b95d28e57127bad5ae76_1.mp3
867
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/cbda882d41bc070e16712364a8f01ef9_1.mp3
868
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/ca829b6bcbce0cb81eb8a20e71c2a164_1.mp3
869
https://www.redkaraoke.com/data/mp3/en/11827/337f73f1ec23e24de8c54d134fa86379_1.mp3
870
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/186d888005d3a07dfaf169a3fa4526f7_1.mp3
871
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/57765c08561613148731e532db27b9d3_1.mp4
872
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/c557856e839d7ca692b70ebe7352e5e3_1.mp4
873
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/03a5d0794ac1b451ce13e069d2da6c23_1.mp3
874
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/4fdc4d850d881a7d435a6b5051933b37_1.mp4
875
https://s3.amazonaws.com/redkaraokecom/

https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/035f3db9ae8dc0ed9719484a6d8d02f2_1.mp3
951
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/e4fad19de673e7aab31f586ec5d7d937_1.mp3
952
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/c04b44c5a884b8fc05e72662faaece4f_1.mp4
953
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/925ae426116078ed3e7f7b4d710ddd5a_1.mp3
954
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/b07322518477af736aadfc6bb7c0fcc2_1.mp4
955
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/3f707e530bbda2ad2130108cba86b218_1.mp3
956
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/b41fbf7530bd61cae7ec62c00f4ebdaf_1.mp3
957
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/dca56ed618429c1bf4a0af95de6a9b84_1.mp4
958
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/9072d93eeaf202b2df485fff075daac3_1.mp4
959
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/9909bb702501374985b2440d0ca7515d_1.mp4
960
https://s3.amazonaws.com/redka

In [8]:
print(len(view_list))

1009


In [9]:
d = {'no_of_views': view_list, 'no_of_likes': like_list, 'no_of_comments': comment_list, 
     'no_of_recordings': recording_list, 'no_of_followers': follower_list, 'no_of_following': following_list,
     'date_recorded': date_list, 'gender': gender_list, 'country': country_list,
     'location': location_list, 'user_avatar': user_avatar_list, 'user_group': user_group_list,
     'user_name': user_name_list, 'language': language_list, 'song_title': song_title_list, 
     'artist': artist_list, 'download_link': download_link_list, 'recording_link': recording_link_list,
     'file_type': file_type_list, 'is_duet': is_duet_list, 'recording_id': recording_id_list}

In [10]:
df = pd.DataFrame(data=d)

In [11]:
df['date_recorded'] = pd.to_datetime(df['date_recorded'])
df['today'] = pd.Timestamp('20171023')
df['days'] = (df['today'] - df['date_recorded']).dt.days
df = df.drop('today',axis=1)

In [12]:
df.head()

,artist,country,date_recorded,download_link,file_type,gender,is_duet,language,location,no_of_comments,...,no_of_likes,no_of_recordings,no_of_views,recording_id,recording_link,song_title,user_avatar,user_group,user_name,days
0,Celine Dion,India,2016-12-22,https://s3.amazonaws.com/redkaraokecom/data/mp...,mp3,NaN,False,en,en,0,...,1,1,6,957137,https://www.redkaraoke.com/profile/Debolina_1/...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/es/img3/n...,2,Debolina_1,305
1,Celine Dion,Kingdom,2017-06-25,https://s3.amazonaws.com/redkaraokecom/data/vi...,vid,woman,False,en,en,0,...,1,1,8,1132634,https://www.redkaraoke.com/profile/CreativeEmm...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,2,CreativeEmma,120
2,Celine Dion,Emirates,2016-12-07,https://s3.amazonaws.com/redkaraokecom/data/mp...,mp3,NaN,False,en,en,0,...,1,1,3,942906,https://www.redkaraoke.com/profile/dingan407/r...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/es/img3/n...,2,dingan407,320
3,Celine Dion,Philippines,2017-06-23,https://s3.amazonaws.com/redkaraokecom/data/vi...,vid,NaN,False,en,en,0,...,1,1,9,1130813,https://www.redkaraoke.com/profile/ginalyngres...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,2,ginalyngresola,122
4,Celine Dion,Ireland,2008-02-15,https://s3.amazonaws.com/redkaraoke/data/mp3/1...,mp3,woman,False,es,en,1,...,1,26,247,99506,https://www.redkaraoke.com/profile/Sari/record...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,2,Sari,3538


In [13]:
df.to_csv('./data/out' + '_' + str(begin_page) + '_' + str(end_page) + '.csv',index=False)